# 여러 문서에서 찾아서 답변하는 챗봇 만들기

In [1]:
!pip install -q langchain langchain-openai openai tiktoken chromadb langchain-community


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [4]:
import requests

url = "https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip"
response = requests.get(url)

with open("data.zip", "wb") as file:
    file.write(response.content)

print("다운로드 완료!")

다운로드 완료!


In [5]:
!unzip data

'unzip'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [6]:
import zipfile

with zipfile.ZipFile("data.zip", "r") as zip_ref:
    zip_ref.extractall("data")

print("압축 해제 완료!")

압축 해제 완료!


## Setting up LangChain

OpenAI API Key

https://platform.openai.com/account/api-keys

In [7]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

## Load multiple and process documents

DirectoryLoader()는 디렉토리 내의 모든 문서들을 로드하는 역할을 합니다. 이때 glob는 해당 디렉토리 내의 어떤 확장자를 가진 파일들을 로드할 것인지를 결정합니다. 예를 들어 *.txt를 매개변수로 사용하면, 해당 디렉토리 내의 모든 txt 파일을 로드합니다. 현재 경로에 있는 모든 txt 파일을 로드하고 txt 파일의 본문은 documents라는 파이썬 리스트의 각 원소로 할당하겠습니다.

In [14]:
# 디렉토리로더는 '주어진 경로에 있는 파일들을 전부 불러와서 로드'
# '.': 현재 경로

loader = DirectoryLoader('./data', glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})  # UTF-8 인코딩 지정)

documents = loader.load()

print('문서의 개수 :', len(documents))

문서의 개수 : 57


파이썬 리스트에 해당하는 documents에서 임의로 1번 문서와 21번 문서를 출력해보겠습니다.

In [15]:
print('1번 문서 :', documents[1])
print('-' * 20)
print('21번 문서 :', documents[21])
print('-' * 20)

1번 문서 : page_content='정책이름: 청년활력 프로그램 운영(청년활동지원센터 운영)
서울시청 미래청년기획단에서 주최하는 일자리 정책은 청년활력을 촉진하기 위해 자율적인 구직활동과 사회참여, 심리/정서지원 등을 제공하여 청년안전망을 구축하는 것을 목표로 합니다. 이 정책의 지원내용은 다음과 같습니다:

마음건강 지원사업: 온라인 고민상담소 'hi, there'를 운영하여 마음건강에 관련된 지원을 제공합니다.

사회참여 지원사업: 청년수당 참여자를 대상으로 오리엔테이션, 자기이해 프로그램, 서울잡스 등의 지원을 통해 사회참여를 도모합니다.

서울청년센터 운영 지원: 서울청년센터의 운영을 지원합니다.

신사업 개발 및 기획사업: 고립청년 개발지원, 홍보 등의 신규 사업 및 기획사업을 개발합니다.

이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 운영되며, 신청자격은 19세부터 39세까지의 서울 거주 청년(청년수당 참여자 포함)입니다. 다양한 연령대를 대상으로 하는 프로그램도 있으며, 학력, 전공, 취업상태, 특화분야에 대한 요건은 없습니다.

신청은 https://sygc.kr/ 사이트에서 각 프로그램별로 개별적으로 접수됩니다. 신청방법, 신청절차, 심사 및 발표에 대한 세부 사항은 공고를 참조하시기 바랍니다. 제출해야 할 서류는 공고에 따라 다를 수 있습니다.

이 정책은 (재)서울현대교육재단과 (사)한국디지털컨버전스협회가 운영합니다. 참고로 https://blog.naver.com/sygcyouth 사이트도 참고하실 수 있습니다.' metadata={'source': 'data\\10.txt'}
--------------------
21번 문서 : page_content='정책제목: 청년과 어르신 주거공유(한지붕세대공감)
서울시청 주택정책과에서는 해당 지역 대학생들의 주거 마련을 위해 할머니와 할아버지들이 남는 방을 학생에게 저렴한 가격에 빌려주는 정책을 운영하고 있습니다. 이 정책은 대학가나 청년들이 많이 사는 지역에서 진행되며, 

documents 리스트의 각 원소의 구조를 살펴보면 page_content의 값으로 본문이 들어가고, metadata 안의 source의 값으로 해당 문서의 원본 파일의 이름이 기재되어져 있습니다.

## Split texts

Langchain의 RecursiveCharacterTextSplitter()는 주어진 문서들을 더 작은 단위의 텍스트 뭉치(chunk)로 잘라주는 역할을 합니다.

chunk_size는 텍스트의 분할 기준이 되는 길이를 설정하고, chunk_overlap은 텍스트를 분할할 경우 텍스트의 내용을 얼만큼 겹치게 할 것인지를 의미합니다. 위에서 총 57개의 문서 파일을 로드한 documents를 입력으로 사용하여 분할된 텍스트 리스트를 texts에 저장합니다.

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수 :', len(texts))

분할된 텍스트의 개수 : 64


57개의 문서가 총 64개의 텍스트로 분할되었습니다. 그 중 임의로 첫번째 문서를 출력해보겠습니다.

In [17]:
texts[0]

Document(metadata={'source': 'data\\1.txt'}, page_content='정책 이름 : 청년예술지원\n서울시청 문화예술과에서는 청년 예술인들을 위한 프로그램을 진행합니다. 이 프로그램은 활동 경력이 부족한 청년 예술인들을 대상으로 하며, 최대 1,000만원의 창작 지원금과 전문가 멘토링 및 네트워킹 프로그램을 제공합니다. 이 프로그램은 2023년 1월 1일부터 2023년 12월 31일까지 운영됩니다. 신청자는 19세에서 39세까지의 나이 조건을 충족하며, 서울에서 첫 작품 발표(오프라인)를 준비 중인 예술인(단체)이어야 합니다. 학력, 전공, 취업 상태, 특화 분야에 대한 요건은 없습니다. 신청은 온라인으로 서울문화예술지원시스템(SCAS)을 통해 진행되며, 신청 기간은 2023년 10월 4일부터 10월 28일까지입니다. 심사 및 발표는 2023년 1월 경에 이루어집니다. 프로그램에 대한 자세한 내용은 https://www.scas.kr/scas/bizinfo/detail/198에서 확인할 수 있으며, 추가 문의는 02-362-9745로 하실 수 있습니다.')

형식은 documents 원소에서 확인하였을 때와 동일하며 page_content에는 분할된 텍스트이 본문이 저장되어져 있고, source에는 해당 본문의 원본 파일의 이름이 저장되어져 있습니다. 57개의 문서가 64개의 텍스트로 분할되었다는 것은 일부 문서는 추가적으로 분할되었음을 의미합니다.  

metadata 안의 source의 값을 이용하여 어떤 문서들을 추가적으로 분할되었는지 확인해보겠습니다. texts의 각 원소들의 source 값을 모두 체크하였을 때 source 값이 중복되어 등장한다면 해당 문서는 추가적으로 분할되었음을 의미합니다.

In [19]:
source_lst = []
for i in range(0, len(texts)):
  source_lst.append(texts[i].metadata['source'])

element_counts = Counter(source_lst)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서 :', filtered_counts)
print('분할된 텍스트의 개수 :', len(documents) + len(filtered_counts))

2개 이상으로 분할된 문서 : {'data\\22.txt': 2, 'data\\23.txt': 2, 'data\\31.txt': 2, 'data\\36.txt': 2, 'data\\40.txt': 2, 'data\\48.txt': 2, 'data\\49.txt': 2}
분할된 텍스트의 개수 : 64


확인 결과 48번 문서, 23번 문서, 31번 문서, 49번 문서, 36번 문서, 22번 문서, 40번 문서가 2개의 텍스트로 추가적으로 분할되었습니다. 총 57개의 문서 중 7개의 문서가 2개의 텍스트로 분할되었음으로 총 합 64개의 텍스트가 생기게 됩니다.

## Create Chroma DB

OpenAI Embedding API를 이용하여 텍스트를 임베딩하고, 코사인 유사도를 통해 유사한 텍스트를 가져오는 실습을 진행한 바 있습니다. Chroma DB는 이 과정들을 기능 별로 이미 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있도록 도와주는 편리한 벡터 응용 도구입니다. Chroma.from_documents()를 통해 벡터 도구 객체를 선언합니다. 이때 documents에는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 사용하고, embedding에는 어떤 종류의 임베딩을 사용할 것인지를 기재해줍니다.

In [20]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)

C:\Users\user\AppData\Local\Temp\ipykernel_10136\3039198533.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


## Make a retriever

벡터 도구 객체를 선언하고 나면 as_retriever()를 통해서 입력된 텍스트로부터 유사한 텍스트를 찾아주는 retriever를 선언할 수 있습니다. retriever를 선언 후 get_relevant_documents()를 통해 입력된 텍스트와 유사한 문서들을 찾아서 반환합니다. 6장에서 실습했던 벡터의 유사도를 구하는 과정을 별도의 추가 구현없이 손쉽게 사용할 수 있습니다.

In [21]:
retriever = vectordb.as_retriever()

In [22]:
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('--' * 20)
print('각 유사 문서의 문서 출처 :')
for doc in docs:
    print(doc.metadata["source"])

C:\Users\user\AppData\Local\Temp\ipykernel_10136\3922651508.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")


유사 문서 개수 : 4
----------------------------------------
첫번째 유사 문서 : page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.

이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.

이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.

이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': 'data\\23.txt'}
----------------------------------------
각 유사 문서의 문서 출처 :
data\23.txt
data\23.txt
data\40.txt
data\39.txt


### 결과를 k개 반환

만약 유사한 문서의 개수를 임의로 지정하고 싶다면 retriever 선언 시에 search_kwargs={"k": <문서의 개수>}를 추가적으로 입력으로 넣어주면 됩니다. 다음의 코드는 유사한 텍스트가 2개만 출력되는 모습을 보여줍니다.

In [23]:
# default k값 = 4
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [24]:
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")

for doc in docs:
    print(doc.metadata["source"])

data\23.txt
data\23.txt


## Make a chain

이제 ChatGPT API와 이미 만들어진 Prompt를 통해서 간단히 챗봇을 구현해봅시다. RetrievalQA.from_chain_type()의 llm 매개변수의 값으로 초기에 임포트한 OpenAI()를 사용할 경우, 기본값으로 ChatGPT API를 사용합니다.chain_type의 매개변수의 값으로 "stuff"를 사용할 경우, 사용자의 눈에는 보이지 않지만 자동으로 아래의 프롬프트를 사용하여 챗봇을 구현합니다.  
```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

{텍스트}에는 사용자의 질문으로부터 높은 유사도를 가진 텍스트가 들어가게 되고, {질문} 부분은 사용자의 질문이 들어가게 됩니다. retriever는 입력된 사용자의 질문으로부터 유사도를 계산하는 도구를 구현하여 매개변수의 값으로 넘겨주면 됩니다. 앞서 이미 Chroma를 통해 벡터 도구 객체로부터 구현한 retreiver를 사용합니다.  

return_source_documents는 챗봇의 답변에 사용된 텍스트들의 출처를 표시할 것인지를 의미합니다. return_source_documents의 값을 False로 할 경우, 챗봇의 답변이 어떤 텍스트에 근거하였는지 알 수 없습니다. 여기서는 뒤에서 근거가 되는 텍스트의 출처를 확인할 것이므로 True를 사용합니다.

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

## Query

이제 qa_chain을 통해 사용자의 입력으로부터 서울 청년 정책과 관련된 챗봇의 답변을 얻을 수 있습니다. 임의의 "대출과 관련된 정책이 궁금합니다"라는 텍스트를 입력하여 qa_chain의 반환 결과를 확인해봅시다.

In [26]:
input_text = "대출과 관련된 정책이 궁금합니다"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

C:\Users\user\AppData\Local\Temp\ipykernel_10136\3234511165.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatbot_response = qa_chain(input_text)


{'query': '대출과 관련된 정책이 궁금합니다', 'result': '서울시에서 운영하는 대출 관련 정책으로는 "서울시 학자금대출 신용회복 지원사업"이 있습니다. 이 정책은 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원을 목적으로 합니다. 지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공하여 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 신청은 서울청년포털을 통해 가능하며, 자세한 내용은 관련 사이트를 참고하시면 됩니다.', 'source_documents': [Document(metadata={'source': 'data\\35.txt'}, page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으로써 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 추가적인 자부담 없이 약정을 체결할 수 있도록 합니다.\n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩니다.\n\n신청은 서울청년포털(youth.seoul.go.kr)을 통해 신청할 수 있으며, 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동할 수 있습니다.\n\n이 정책은 서울시 미래청년기획단이 운영하며, 자세한 내용은 관련 사이트를 참고하시기 바랍니다.\n\n[참고사이트]\n신용회복 신청 안내: https://youth.seoul.go.kr/site/main/board/notice/27789?baCategory1=basic&baCommSelec=true\n신청사이트: 

qa_chain의 결과를 chatbot_response에 저장하고 출력하면 'query', 'result', source_documents 세 개의 key값을 갖고 있습니다. 이 중 source_documents의 경우 내부적으로 앞서 확인한 형식인 page_content와 meta_data의 값을 가집니다.

'query'의 경우 현재 입력된 사용자의 질문을 의미합니다. 이때 qa_chain은 앞서 구현한 retriever을 통해서  내부적으로 '대출과 관련된 정책이 궁금합니다'와 유사도가 높은 텍스트 2개를 찾아냅니다. metadata 내의 source의 값에 따르면 해당 텍스트는 35.txt 파일과 39.txt 파일의 본문에 해당되며 두  본문(여기서는 page_content의 값)을 통해서 앞서 위에서 확인한 stuff 프롬프트를 통하여 챗봇은 답변을 작성합니다. 해당 챗봇의 답변은 결과적으로 'result'의 값에 해당됩니다.

현재 chatbot_response의 값은 너무 복잡합니다. 실제 사용자가 사용하는 챗봇을 가정하기 위해 chatbot_response로부터 실제 챗봇 답변에 해당하는 'result' 부분만 꺼내오고, 또한 답변에 사용된 출처를 확인하기 위해 metadata 내의 source의 값을 추가적으로 출력하는 get_chatbot_response() 함수를 구현합니다.

In [27]:
# result(답변)만 가져오는 함수 정의
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['source'])

"신혼 부부의 신혼집 마련을 위한 정책이 있을까?"라는 임의의 입력에 대해서 챗봇의 답변을 확인해보겠습니다.

In [28]:
input_text = "신혼 부부의 신혼집 마련을 위한 정책이 있을까?"
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 서울시에서는 신혼부부를 위한 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련되었습니다. 지원 대상은 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정인 자로, 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부입니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 대출은 임차보증금의 90% 이내 또는 2억원 중 작은 금액으로 지원되며, 최대 연 3.6% 이차보전 및 최장 10년까지 지원됩니다. 신청은 서울주거포털을 통해 온라인으로 가능합니다.

문서 출처:
data\23.txt
data\23.txt


신혼부부 임차 보증금 지원 정책을 소개합니다. 텍스트의 출처가 23.txt가 두 번 출력됩니다. 이는 앞서 text_splitter.split_documents()가 57개의 문서를 64개의 텍스트로 분할하는 가정에서 23.txt가 두 개의 텍스트로 분할되었기 때문입니다. chatbot_response의 값을 출력해보겠습니다.

In [29]:
# print(chatbot_response)
chatbot_response

{'query': '신혼 부부의 신혼집 마련을 위한 정책이 있을까?',
 'result': '네, 서울시에서는 신혼부부를 위한 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련되었습니다. 지원 대상은 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정인 자로, 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부입니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 대출은 임차보증금의 90% 이내 또는 2억원 중 작은 금액으로 지원되며, 최대 연 3.6% 이차보전 및 최장 10년까지 지원됩니다. 신청은 서울주거포털을 통해 온라인으로 가능합니다.',
 'source_documents': [Document(metadata={'source': 'data\\23.txt'}, page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한

출처가 23.txt로 동일하지만 두 개의 텍스트는 서로 다릅니다. 원본 파일 23.txt가 두 개의 텍스트로 분할되었기 때문입니다. 또한 두 개의 텍스트에서 **'이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.'** 라는 텍스트가 공통적으로 들어가있음을 확인할 수 있습니다. 이는 RecursiveCharacterTextSplitter() 선언 시에 chunk_overlap을 사용하였기 때문에 텍스트를 분할할 때 겹치지 않도록 분할하는 것이 아니라 일부 겹칠 수 있도록 허용하였기 때문입니다.

이번에는 실제 챗봇과의 답변을 가정하고 사용자의 질문으로부터 챗봇의 답변이 오면 해당 챗봇의 답변으로부터 이어서 사용자가 질문하는 시나리오를 진행해보겠습니다. "전세 자금이 부족한 사람을 위한 정책 이름이 뭐야?"라는 임의의 질문을 입력합니다.

In [30]:
input_text = "전세자금이 부족한 사람을 위한 정책 이름이 뭐야?"
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

죄송하지만, 전세자금이 부족한 사람을 위한 특정 정책 이름에 대한 정보는 제공되지 않았습니다. 다른 정책이나 지원 프로그램에 대한 정보가 필요하시면 추가로 문의해 주세요.

문서 출처:
data\40.txt
data\40.txt


'희망두배 청년통장'이라는 정책이 있음을 알았습니다. 이어서 '희망두배 청년통장'의 구체적인 지원 내용을 문의합니다.

In [31]:
query = "희망두배 청년통장은 어떤걸 지원하니?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장은 저소득 근로 청년들의 자립을 지원하기 위해 서울시청 안심돌봄복지과에서 운영하는 정책입니다. 이 정책은 교육비, 주거비, 결혼자금, 창업운영자금 등을 위한 저축을 지원합니다. 

지원 내용은 다음과 같습니다:
- 저축 목적에 따라 월 10만원 또는 15만원의 저축액을 선택할 수 있으며, 저축기간은 2년 또는 3년입니다.
- 본인이 적립한 금액의 100%를 지원받을 수 있으며, 10만원 저축 시 2년형은 480만원 이상, 3년형은 720만원 이상, 15만원 저축 시 2년형은 720만원 이상, 3년형은 1,080만원 이상의 지원이 이루어집니다.

신청 자격은 18세에서 34세까지의 서울시 거주자로, 공고일 현재 근로 중인 자격 요건을 모두 충족하는 경우에 신청할 수 있습니다.

문서 출처:
data\40.txt
data\36.txt


이번에는 지원 조건을 문의해봅시다.

In [32]:
query = "희망두배 청년통장의 지원 조건은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 지원 조건은 다음과 같습니다:

1. 연령: 18세에서 34세까지
2. 거주지: 서울시 거주자
3. 근로 상태: 공고일 현재 근로 중인 자

이 조건을 모두 충족하는 경우에 신청할 수 있습니다.

문서 출처:
data\40.txt
data\36.txt


희망두배 청년통장의 마감 기한을 문의해봅시다.

In [33]:
query = "희망두배 청년통장의 마감 기한은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

죄송하지만, 제공된 정보에는 희망두배 청년통장의 마감 기한에 대한 구체적인 내용이 포함되어 있지 않습니다. 자세한 신청 기한은 서울시복지재단의 홈페이지나 서울시 안심돌봄복지과의 홈페이지에서 확인하시기 바랍니다.

문서 출처:
data\40.txt
data\37.txt


원본 문서에서 마감 기한이 구체적으로 나와있지 않아 마감 기한을 알 수 없다고 답변합니다. 해당 챗봇은 주어진 57개의 문서를 기반으로 답변하기 때문입니다. 이렇게 다수의 문서로부터 질의 응답을 할 수 있는 챗봇을 구현해보았습니다. 이렇게 구현한 챗봇을 앞서 실습과 같이 streamlit으로 UI를 구현하거나, 카카오톡이나 텔레그램 등을 연동하여 나만의 커스텀 챗봇을 구현할 수 있습니다.

In [34]:
!pip install gradio


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


To create a public link, set `share=True` in `launch()`.  
Running on https://localhost:7860/  

라는 식의 문구가 나오면 위의 localhost로 시작하는 주소를 클릭하세요.

In [35]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="청년정책챗봇") # 청년정책챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']
      bot_message += ' # sources :'

      # 답변의 출처를 표기
      for i, doc in enumerate(result['source_documents']):
          bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + ' '

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

C:\ProgramData\anaconda3\Lib\site-packages\gradio\components\chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
